# Load packages

In [ ]:
# Load libraries
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#from fastai.vision.all import *
import torch
#from ipywidgets import IntProgress
import glob
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import cv2, os
from pathlib import Path
from sklearn.metrics import jaccard_score, f1_score, precision_score, recall_score
import random
from skimage.transform import resize
from PIL import Image
import matplotlib.transforms as mtransforms

In [ ]:
from PIL import Image
import matplotlib.transforms as mtransforms

# UNET18

In [ ]:
def metrics(pred_path_list, target_path_dir, num_classes):
    metrics_dicts = {"precision": [], "recall": [], "jaccard": [], "macrof1": []}

    for i in pred_path_list:
        # Compare all image results from the test dataset
        filename = os.path.basename(i).replace('npy', 'png').replace("prediction", "mask")

        # Load the images as np arrays
        targ = np.array(Image.open(f"{target_path_dir}/{filename}"))
        pred = np.load(i)
    
        # Flatten them into vectors
        ytrue = targ.flatten()
        ypred = pred.flatten()

        # Precision metrics
        metrics_dicts["precision"].append(precision_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["recall"].append(recall_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["jaccard"].append(jaccard_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["macrof1"].append(f1_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        

    # Now get the mean value 
    for metric_key in metrics_dicts.keys():
        # Remove samples that did not have the class predicted
        clean_metric = [value for value in metrics_dicts[metric_key] if np.any(value != 0)]
        
        # Check if the clean_metric list is not empty before calculating the mean
        if clean_metric:
            metrics_dicts[metric_key] = np.mean(clean_metric)
        else:
            metrics_dicts[metric_key] = 0.0

    return metrics_dicts


In [ ]:
#might need to rename files if not case sensitive
York_Canola = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unet18/unet18_preds/unet18fold*_predictions/YC*")
T1_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unet18/unet18_preds/unet18fold*_predictions/*t1_prediction*")
T2_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unet18/unet18_preds/unet18fold*_predictions/*t2_prediction*")


In [ ]:
num_classes=3
York_Canola_dict = metrics(York_Canola, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels", num_classes)
T1_Miling_dict = metrics(T1_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels", num_classes)
T2_Miling_dict = metrics(T2_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels", num_classes)

In [ ]:
dataset = ["York_Canola", "T1_Miling_ARG", "T2_Miling_ARG"]

In [ ]:
# make a dataframe with the results
#kfold2 csv made with new holdout set
results_df = pd.DataFrame(
    data=[York_Canola_dict, T1_Miling_dict, T2_Miling_dict],
    index=dataset)

# save the results as csv
results_df.to_csv("unet18_metric_results.csv")
results_df

In [ ]:
a = pd.read_csv("unet18_metric_results.csv")
a.mean()
#incorrect codes file so predicting the wrong thing?
#are labels in test set correct? 

In [ ]:
x = []
fig, axes = plt.subplots(2, 3, figsize=(12, 8))  # Create a 2x3 grid for subplots

for i in range(6):
    x.append(np.load(f'/group/pawsey0149/mmckay/singularity/unet_results/unet18/1fold_predicted_holdoutset/C4t1_prediction_2.npy'))
    row = i // 3  # Calculate the row index
    col = i % 3   # Calculate the column index
    axes[row, col].imshow(x[i])  # Plot the image in the corresponding subplot

plt.tight_layout()  # Adjust the spacing between subplots
plt.show()

In [ ]:
def remove_class(img, mask, obj_class=0):
    removed = np.zeros(img.shape, dtype='uint8')

    for i in range(img.shape[2]):
        band = img[:,:,i]
        band = np.where(mask == obj_class, 0, band)
        removed[:,:, i] = band
    return removed

In [ ]:
plt.rcParams.update({'font.family':'sans-serif',
              'font.sans-serif': 'Arial'})

f, axs = plt.subplots(1,3, figsize=(30,20))

part0 = np.array(Image.open('/group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/C4t1_image_2.png'))
part1 = np.array(Image.open('/group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/C4t1_image_6.png'))
concat_im = np.concatenate((part0,part1), axis=0)


p0 = np.load("/group/pawsey0149/mmckay/singularity/unet_results/unet18/1fold_predicted_holdoutset/C4t1_prediction_2.npy")
p1 = np.load("/group/pawsey0149/mmckay/singularity/unet_results/unet18/1fold_predicted_holdoutset/C4t1_prediction_6.npy")
concat_pred = np.concatenate((p0,p1), axis=0)

part0 = np.array(Image.open('/group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/C4t1_mask_2.png'))
part1 = np.array(Image.open('//group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/C4t1_mask_6.png'))
concat_label = np.concatenate((part0,part1), axis=0)

# Plot the RGB overlaid with masks   
b = remove_class(concat_im, concat_pred, obj_class=0)
bl = remove_class(concat_im, concat_label, obj_class=0)

axs[0].imshow(concat_im[:,:340,:])
axs[1].imshow(bl[:,:340])
axs[1].imshow(concat_label[:,:340], alpha=0.3)

axs[2].imshow(b[:,:340])
axs[2].imshow(concat_pred[:,:340], alpha  =0.3)

# axs.imshow(np.load(), alpha=0.5)
# axs.suptitle('test title')

In [ ]:
plt.rcParams.update({'font.family':'sans-serif',
              'font.sans-serif': 'Arial'})

f, axs = plt.subplots(1,3, figsize=(30,20))

part0 = np.array(Image.open('/group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/Cund_T1_5008_image_0.png'))
part1 = np.array(Image.open('/group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/D12t2_image_5.png'))
concat_im = np.concatenate((part0,part1), axis=0)


p0 = np.load("/group/pawsey0149/mmckay/singularity/unet_results/unet18/1fold_predicted_holdoutset/Cund_T1_5008_prediction_0.npy")
p1 = np.load("/group/pawsey0149/mmckay/singularity/unet_results/unet18/1fold_predicted_holdoutset/D12t2_prediction_5.npy")
concat_pred = np.concatenate((p0,p1), axis=0)

part0 = np.array(Image.open('/group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/Cund_T1_5008_mask_0.png'))
part1 = np.array(Image.open('//group/pawsey0149/mmckay/singularity/train_val_sets_PNG/images_labels_new/D12t2_mask_5.png'))
concat_label = np.concatenate((part0,part1), axis=0)

# Plot the RGB overlaid with masks   
b = remove_class(concat_im, concat_pred, obj_class=0)
bl = remove_class(concat_im, concat_label, obj_class=0)

axs[0].imshow(concat_im[:,:340,:])
axs[1].imshow(bl[:,:340])
axs[1].imshow(concat_label[:,:340], alpha=0.3)

axs[2].imshow(b[:,:340])
axs[2].imshow(concat_pred[:,:340], alpha  =0.3)

# axs.imshow(np.load(), alpha=0.5)
# axs.suptitle('test title')

# UNET34

In [ ]:
def metrics(pred_path_list, target_path_dir, num_classes):
    metrics_dicts = {"precision": [], "recall": [], "jaccard": [], "macrof1": []}

    for i in pred_path_list:
        # Compare all image results from the test dataset
        filename = os.path.basename(i).replace('npy', 'png').replace("prediction", "mask")

        # Load the images as np arrays
        targ = np.array(Image.open(f"{target_path_dir}/{filename}"))
        pred = np.load(i)

        # Flatten them into vectors
        ytrue = targ.flatten()
        ypred = pred.flatten()

        # Precision metrics
        metrics_dicts["precision"].append(precision_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["recall"].append(recall_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["jaccard"].append(jaccard_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["macrof1"].append(f1_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        

    # Now get the mean value 
    for metric_key in metrics_dicts.keys():
        # Remove samples that did not have the class predicted
        clean_metric = [value for value in metrics_dicts[metric_key] if np.any(value != 0)]
        
        # Check if the clean_metric list is not empty before calculating the mean
        if clean_metric:
            metrics_dicts[metric_key] = np.mean(clean_metric)
        else:
            metrics_dicts[metric_key] = 0.0

    return metrics_dicts

In [ ]:

York_Canola = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unet34/unet34_preds/unet34fold*_predictions/YC*")
T1_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unet34/unet34_preds/unet34fold*_predictions/*t1_prediction*")
T2_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unet34/unet34_preds/unet34fold*_predictions/*t2_prediction*")


In [ ]:
num_classes=3
York_Canola_dict = metrics(York_Canola, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels", num_classes)
T1_Miling_dict = metrics(T1_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels", num_classes)
T2_Miling_dict = metrics(T2_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels", num_classes)

In [ ]:
dataset = ["York_Canola", "T1_Miling_ARG", "T2_Miling_ARG"]

In [ ]:
# make a dataframe with the results
results_df = pd.DataFrame(
    data=[York_Canola_dict, T1_Miling_dict, T2_Miling_dict],
    index=dataset)

# save the results as csv
results_df.to_csv("unet34_metric_results.csv")
results_df

In [ ]:
x = []
fig, axes = plt.subplots(2, 3, figsize=(12, 8))  # Create a 2x3 grid for subplots

for i in range(6):
    x.append(np.load(f'/group/pawsey0149/mmckay/singularity/unet_results/unet34/1fold_predicted_holdoutset/A5t1_prediction_11.npy'))
    row = i // 3  # Calculate the row index
    col = i % 3   # Calculate the column index
    axes[row, col].imshow(x[i])  # Plot the image in the corresponding subplot

plt.tight_layout()  # Adjust the spacing between subplots
plt.show()

# VGG16

In [ ]:
#need to resize ground truth images from 500 by 500 pixels to 224 by 224 to account for modifcation to epxorted image size due to 
#Vram memory constraints 

# Set the input directory containing the 500x500 images
input_directory = "/group/pawsey0149/mmckay/singularity/images_labels_all/images"

# Set the output directory where resized images will be saved
output_directory = "/group/pawsey0149/mmckay/singularity/images_labels_all/images224"

# Ensure the output directory exists, or create it if necessary
os.makedirs(output_directory, exist_ok=True)

# Set the target size (224x224 pixels)
target_size = (224, 224)

# List all files in the input directory
image_files = [f for f in os.listdir(input_directory) if f.endswith((".jpg", ".jpeg", ".png", ".bmp"))]

# Loop through each image file and resize
for image_file in image_files:
    # Open the image
    img_path = os.path.join(input_directory, image_file)
    img = Image.open(img_path)

    # Resize the image to the target size
    img_resized = img.resize(target_size, Image.ANTIALIAS)

    # Save the resized image to the output directory
    output_path = os.path.join(output_directory, image_file)
    img_resized.save(output_path)

    print(f"Resized {image_file} to {target_size}")

print("Resize completed.")

In [ ]:
#need to resize ground truth images from 500 by 500 pixels to 224 by 224 to account for modifcation to epxorted image size due to 
#Vram memory constraints 

# Set the input directory containing the 500x500 images
input_directory = "/group/pawsey0149/mmckay/singularity/images_labels_all/labels"

# Set the output directory where resized images will be saved
output_directory = "/group/pawsey0149/mmckay/singularity/images_labels_all/labels224"

# Ensure the output directory exists, or create it if necessary
os.makedirs(output_directory, exist_ok=True)

# Set the target size (224x224 pixels)
target_size = (224, 224)

# List all files in the input directory
image_files = [f for f in os.listdir(input_directory) if f.endswith((".jpg", ".jpeg", ".png", ".bmp"))]

# Loop through each image file and resize
for image_file in image_files:
    # Open the image
    img_path = os.path.join(input_directory, image_file)
    img = Image.open(img_path)

    # Resize the image to the target size
    img_resized = img.resize(target_size, Image.ANTIALIAS)

    # Save the resized image to the output directory
    output_path = os.path.join(output_directory, image_file)
    img_resized.save(output_path)

    print(f"Resized {image_file} to {target_size}")

print("Resize completed.")

In [ ]:
#introduced shape test and discrete class labels for this model
def metrics(pred_path_list, target_path_dir, num_classes):
    metrics_dicts = {"precision": [], "recall": [], "jaccard": [], "macrof1": []}

    for i in pred_path_list:
        # Compare all image results from the test dataset
        filename = os.path.basename(i).replace('npy', 'png').replace("prediction", "mask")

        # Load the images as np arrays
        targ = np.array(Image.open(f"{target_path_dir}/{filename}"))
        pred = np.load(i)

        # Flatten them into vectors
        ytrue = targ.flatten()
        ypred = pred.flatten()

        # Precision metrics
        metrics_dicts["precision"].append(precision_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["recall"].append(recall_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["jaccard"].append(jaccard_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        metrics_dicts["macrof1"].append(f1_score(ytrue, ypred, average=None, zero_division=0, labels=range(num_classes)))
        

    # Now get the mean value 
    for metric_key in metrics_dicts.keys():
        # Remove samples that did not have the class predicted
        clean_metric = [value for value in metrics_dicts[metric_key] if np.any(value != 0)]
        
        # Check if the clean_metric list is not empty before calculating the mean
        if clean_metric:
            metrics_dicts[metric_key] = np.mean(clean_metric)
        else:
            metrics_dicts[metric_key] = 0.0

    return metrics_dicts

In [ ]:

York_Canola = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unetvgg/unetvgg_preds/unetVGGfold*_predictions/YC*")
T1_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unetvgg/unetvgg_preds/unetVGGfold*_predictions/*t1_prediction*")
T2_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results/unetvgg/unetvgg_preds/unetVGGfold*_predictions/*t2_prediction*")


In [ ]:
York_Canola

In [ ]:
num_classes=3
York_Canola_dict = metrics(York_Canola, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels224", num_classes)
T1_Miling_dict = metrics(T1_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels224", num_classes)
T2_Miling_dict = metrics(T2_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all/labels224", num_classes)

In [ ]:
dataset = ["York_Canola", "T1_Miling_ARG", "T2_Miling_ARG"]


In [ ]:
# make a dataframe with the results
results_df = pd.DataFrame(
    data=[York_Canola_dict, T1_Miling_dict, T2_Miling_dict],
    index=dataset)

# save the results as csv
results_df.to_csv("unetvgg_metric_results.csv")
results_df

# VGG16 224 pixel raw images

In [ ]:
York_Canola = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results_224s/unetvgg/unetvgg_preds/unetVGGfold*_predictions/YC*")
T1_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results_224s/unetvgg/unetvgg_preds/unetVGGfold*_predictions/*t1_prediction*")
T2_Miling_ARG = glob.glob("/group/pawsey0149/mmckay/singularity/unet_results_224s/unetvgg/unetvgg_preds/unetVGGfold*_predictions/*t2_prediction*")


In [ ]:
York_Canola

In [ ]:
num_classes=3
York_Canola_dict = metrics(York_Canola, "/group/pawsey0149/mmckay/singularity/images_labels_all_224/labels", num_classes)
T1_Miling_dict = metrics(T1_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all_224/labels", num_classes)
T2_Miling_dict = metrics(T2_Miling_ARG, "/group/pawsey0149/mmckay/singularity/images_labels_all_224/labels", num_classes)

In [ ]:
dataset = ["York_Canola", "T1_Miling_ARG", "T2_Miling_ARG"]

In [ ]:
# make a dataframe with the results
results_df = pd.DataFrame(
    data=[York_Canola_dict, T1_Miling_dict, T2_Miling_dict],
    index=dataset)

# save the results as csv
results_df.to_csv("unetvgg224raw_metric_results.csv")
results_df

# Plotting unet34 results

In [ ]:
def remove_class(img, mask, obj_class=0):
    removed = np.zeros(img.shape, dtype='uint8')

    for i in range(img.shape[2]):
        band = img[:,:,i]
        band = np.where(mask == obj_class, 0, band)
        removed[:,:, i] = band
    return removed

In [ ]:
# Define the display images
display_im = [
    "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/D6t2_prediction_2.npy",
    "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/E10t2_prediction_7.npy",
    "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/F3t2_prediction_8.npy"
]
path = "/scratch/pawsey0149/mmckay/holdout_training_set"

# Create the subplots
fig, axs = plt.subplots(3, 3, figsize=(40, 40))
axs = axs.ravel()

# Define labels for columns
column_labels = ['A', 'B', 'C']

# Define labels for rows
row_labels = ['RGB image', 'Ground truth', 'Prediction']

for i, filepath in enumerate(display_im):
    # Get image and ground truth file names
    image_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "image")
    label_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "mask")
    
    # Load the images
    im = np.array(Image.open(f"{path}/{image_name}"))
    targ = np.array(Image.open(f"{path}/{label_name}"))
    pred = np.load(filepath)
    
    # Mask the soil out
    maskedT = remove_class(im, targ, obj_class=0)
    maskedP = remove_class(im, pred, obj_class=0)
    
    # RGB image
    from matplotlib.colors import ListedColormap
    colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])
    
    # Set column titles
    axs[i].set_title(column_labels[i], fontsize=60)
    
    # Set row titles for the first column
    if i % 3 == 0:
        axs[i].text(-0.1, 0.5, row_labels[i // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i].transAxes)
    
    # Base image
    axs[i].imshow(im)
    
    # Set row titles for the second column
    if (i+3) % 3 == 0:
        axs[i + 3].text(-0.1, 0.5, row_labels[(i+3) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 3].transAxes)
    
    # Ground truth
    axs[i + 3].imshow(im)
    axs[i + 3].imshow(maskedT)
    axs[i + 3].imshow(targ, cmap=colors, alpha=0.5)
    
    # Set row titles for the third column
    if (i+6) % 3 == 0:
        axs[i + 6].text(-0.1, 0.5, row_labels[(i+6) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 6].transAxes)
    
    # Prediction
    axs[i + 6].imshow(im)
    axs[i + 6].imshow(maskedP)
    axs[i + 6].imshow(pred, cmap=colors, alpha=0.5)

for p in range(9):
    axs[p].axis("off")

# Set the main title for the entire plot
fig.suptitle("T2 Miling", fontsize=80)

# Adjust subplot layout to fit titles properly
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save or show the plot
plt.savefig('t2_miling.png', dpi=500)
plt.show()

In [ ]:
# Define the display images
display_im = ["/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/B7t1_prediction_8.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/C4t1_prediction_10.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/F5t1_prediction_0.npy"]
path = "/scratch/pawsey0149/mmckay/holdout_training_set"


# Create the subplots
fig, axs = plt.subplots(3, 3, figsize=(40, 40))
axs = axs.ravel()

# Define labels for columns
column_labels = ['A', 'B', 'C']

# Define labels for rows
row_labels = ['RGB image', 'Ground truth', 'Prediction']

for i, filepath in enumerate(display_im):
    # Get image and ground truth file names
    image_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "image")
    label_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "mask")
    
    # Load the images
    im = np.array(Image.open(f"{path}/{image_name}"))
    targ = np.array(Image.open(f"{path}/{label_name}"))
    pred = np.load(filepath)
    
    # Mask the soil out
    maskedT = remove_class(im, targ, obj_class=0)
    maskedP = remove_class(im, pred, obj_class=0)
    
    # RGB image
    from matplotlib.colors import ListedColormap
    colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])
    
    # Set column titles
    axs[i].set_title(column_labels[i], fontsize=60)
    
    # Set row titles for the first column
    if i % 3 == 0:
        axs[i].text(-0.1, 0.5, row_labels[i // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i].transAxes)
    
    # Base image
    axs[i].imshow(im)
    
    # Set row titles for the second column
    if (i+3) % 3 == 0:
        axs[i + 3].text(-0.1, 0.5, row_labels[(i+3) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 3].transAxes)
    
    # Ground truth
    axs[i + 3].imshow(im)
    axs[i + 3].imshow(maskedT)
    axs[i + 3].imshow(targ, cmap=colors, alpha=0.5)
    
    # Set row titles for the third column
    if (i+6) % 3 == 0:
        axs[i + 6].text(-0.1, 0.5, row_labels[(i+6) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 6].transAxes)
    
    # Prediction
    axs[i + 6].imshow(im)
    axs[i + 6].imshow(maskedP)
    axs[i + 6].imshow(pred, cmap=colors, alpha=0.5)

for p in range(9):
    axs[p].axis("off")

# Set the main title for the entire plot
fig.suptitle("T1 Miling", fontsize=80)

# Adjust subplot layout to fit titles properly
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save or show the plot
plt.savefig('t1_miling.png', dpi=500)
plt.show()

In [ ]:
# Define the display images
display_im = ["/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_106_prediction_2.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_114_prediction_13.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_172_prediction_3.npy"]
path = "/scratch/pawsey0149/mmckay/holdout_training_set"


# Create the subplots
fig, axs = plt.subplots(3, 3, figsize=(40, 40))
axs = axs.ravel()

# Define labels for columns
column_labels = ['A', 'B', 'C']

# Define labels for rows
row_labels = ['RGB image', 'Ground truth', 'Prediction']

for i, filepath in enumerate(display_im):
    # Get image and ground truth file names
    image_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "image")
    label_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "mask")
    
    # Load the images
    im = np.array(Image.open(f"{path}/{image_name}"))
    targ = np.array(Image.open(f"{path}/{label_name}"))
    pred = np.load(filepath)
    
    # Mask the soil out
    maskedT = remove_class(im, targ, obj_class=0)
    maskedP = remove_class(im, pred, obj_class=0)
    
    # RGB image
    from matplotlib.colors import ListedColormap
    colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])
    
    # Set column titles
    axs[i].set_title(column_labels[i], fontsize=60)
    
    # Set row titles for the first column
    if i % 3 == 0:
        axs[i].text(-0.1, 0.5, row_labels[i // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i].transAxes)
    
    # Base image
    axs[i].imshow(im)
    
    # Set row titles for the second column
    if (i+3) % 3 == 0:
        axs[i + 3].text(-0.1, 0.5, row_labels[(i+3) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 3].transAxes)
    
    # Ground truth
    axs[i + 3].imshow(im)
    axs[i + 3].imshow(maskedT)
    axs[i + 3].imshow(targ, cmap=colors, alpha=0.5)
    
    # Set row titles for the third column
    if (i+6) % 3 == 0:
        axs[i + 6].text(-0.1, 0.5, row_labels[(i+6) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 6].transAxes)
    
    # Prediction
    axs[i + 6].imshow(im)
    axs[i + 6].imshow(maskedP)
    axs[i + 6].imshow(pred, cmap=colors, alpha=0.5)

for p in range(9):
    axs[p].axis("off")

# Set the main title for the entire plot
fig.suptitle("York Canola", fontsize=80)

# Adjust subplot layout to fit titles properly
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save or show the plot
plt.savefig('york_canola1.png', dpi=500)
plt.show()

In [ ]:
# Define the display images
display_im = ["/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_142_prediction_6.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_151_prediction_10.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_59_prediction_12.npy"]
path = "/scratch/pawsey0149/mmckay/holdout_training_set"


# Create the subplots
fig, axs = plt.subplots(3, 3, figsize=(40, 40))
axs = axs.ravel()

# Define labels for columns
column_labels = ['A', 'B', 'C']

# Define labels for rows
row_labels = ['RGB image', 'Ground truth', 'Prediction']

for i, filepath in enumerate(display_im):
    # Get image and ground truth file names
    image_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "image")
    label_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "mask")
    
    # Load the images
    im = np.array(Image.open(f"{path}/{image_name}"))
    targ = np.array(Image.open(f"{path}/{label_name}"))
    pred = np.load(filepath)
    
    # Mask the soil out
    maskedT = remove_class(im, targ, obj_class=0)
    maskedP = remove_class(im, pred, obj_class=0)
    
    # RGB image
    from matplotlib.colors import ListedColormap
    colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])
    
    # Set column titles
    axs[i].set_title(column_labels[i], fontsize=60)
    
    # Set row titles for the first column
    if i % 3 == 0:
        axs[i].text(-0.1, 0.5, row_labels[i // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i].transAxes)
    
    # Base image
    axs[i].imshow(im)
    
    # Set row titles for the second column
    if (i+3) % 3 == 0:
        axs[i + 3].text(-0.1, 0.5, row_labels[(i+3) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 3].transAxes)
    
    # Ground truth
    axs[i + 3].imshow(im)
    axs[i + 3].imshow(maskedT)
    axs[i + 3].imshow(targ, cmap=colors, alpha=0.5)
    
    # Set row titles for the third column
    if (i+6) % 3 == 0:
        axs[i + 6].text(-0.1, 0.5, row_labels[(i+6) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 6].transAxes)
    
    # Prediction
    axs[i + 6].imshow(im)
    axs[i + 6].imshow(maskedP)
    axs[i + 6].imshow(pred, cmap=colors, alpha=0.5)

for p in range(9):
    axs[p].axis("off")

# Set the main title for the entire plot
fig.suptitle("York Canola", fontsize=80)

# Adjust subplot layout to fit titles properly
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save or show the plot
plt.savefig('york_canola2.png', dpi=500)
plt.show()

In [ ]:
# Define the display images
display_im = ["/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/B7t1_prediction_8.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/E10t2_prediction_7.npy",
              "/scratch/pawsey0149/mmckay/unet_results/unet34/unet34_preds/unet34fold0_predictions/YC_IMG_114_prediction_13.npy"]
path = "/scratch/pawsey0149/mmckay/holdout_training_set"


# Create the subplots
fig, axs = plt.subplots(3, 3, figsize=(40, 40))
axs = axs.ravel()

# Define labels for columns
column_labels = ['T1 Miling', 'T2 Miling', 'York Canola']

# Define labels for rows
row_labels = ['RGB image', 'Ground truth', 'Prediction']

for i, filepath in enumerate(display_im):
    # Get image and ground truth file names
    image_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "image")
    label_name = os.path.basename(filepath).replace('npy', 'png').replace("prediction", "mask")
    
    # Load the images
    im = np.array(Image.open(f"{path}/{image_name}"))
    targ = np.array(Image.open(f"{path}/{label_name}"))
    pred = np.load(filepath)
    
    # Mask the soil out
    maskedT = remove_class(im, targ, obj_class=0)
    maskedP = remove_class(im, pred, obj_class=0)
    
    # RGB image
    from matplotlib.colors import ListedColormap
    colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])
    
    # Set column titles
    axs[i].set_title(column_labels[i], fontsize=60)
    
    # Set row titles for the first column
    if i % 3 == 0:
        axs[i].text(-0.1, 0.5, row_labels[i // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i].transAxes)
    
    # Base image
    axs[i].imshow(im)
    
    # Set row titles for the second column
    if (i+3) % 3 == 0:
        axs[i + 3].text(-0.1, 0.5, row_labels[(i+3) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 3].transAxes)
    
    # Ground truth
    axs[i + 3].imshow(im)
    axs[i + 3].imshow(maskedT)
    axs[i + 3].imshow(targ, cmap=colors, alpha=0.5)
    
    # Set row titles for the third column
    if (i+6) % 3 == 0:
        axs[i + 6].text(-0.1, 0.5, row_labels[(i+6) // 3], fontsize=60, rotation=90, va='center', ha='center', transform=axs[i + 6].transAxes)
    
    # Prediction
    axs[i + 6].imshow(im)
    axs[i + 6].imshow(maskedP)
    axs[i + 6].imshow(pred, cmap=colors, alpha=0.5)

for p in range(9):
    axs[p].axis("off")

# Set the main title for the entire plot
fig.suptitle("All Datasets", fontsize=80)

# Adjust subplot layout to fit titles properly
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save or show the plot
plt.savefig('combined.png', dpi=300)
plt.show()

# Plotting Image processing

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os

# Define the colormap
colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])

# Create subplots as a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(50, 50))
axs = axs.ravel()

# Set the title
fig.suptitle("Image Processing", fontsize=10)

# Define the specific image files to use
image_files = ['YC_IMG_35_image.npy', 'YC_IMG_123_image.npy']

# Import and display the specified raw images
for i in range(2):
    filename = image_files[i]
    im = np.load(f'/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/{filename}')
    axs[i].imshow(im, alpha=0.9, cmap=colors)  # Apply the colormap
    axs[i].set_title(os.path.splitext(filename)[0], fontsize=80)  # Remove .npy extension
    axs[i].axis("off")
    
    # Find the corresponding soil crop mask file
    crop_mask_file = f'/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/{filename.replace("image", "soil_crop_mask")}'
    crop_mask = np.load(crop_mask_file)
    axs[i + 2].imshow(crop_mask, alpha=0.9, cmap=colors)  # Apply the colormap to the crop mask
    axs[i + 2].set_title(os.path.splitext(os.path.basename(crop_mask_file))[0], fontsize=80)  # Remove .npy extension
    axs[i + 2].axis("off")

plt.tight_layout()

# Save or show the plot
plt.savefig('Image_processing.png', dpi=300)
plt.show()




In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import random
import glob

# Define the colormap
colors = ListedColormap(["#000000", "#72ccd1", "#ff7d00"])

# Create subplots as a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(50, 50))
axs = axs.ravel()

# Set the title
fig.suptitle("Image Processing", fontsize=15)

# Load image files
image_files = glob.glob('/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/*image.npy')

# Import and display 2 random raw images
for i in range(2):
    filename = os.path.basename(random.choice(image_files))
    im = np.load(f'/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/{filename}')
    axs[i].imshow(im, alpha=0.9, cmap=colors)  # Apply the colormap
    axs[i].set_title(filename, fontsize=60)
    axs[i].axis("off")
    
     # Find the corresponding soil crop mask file
    crop_mask_file = f'/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/{filename.replace("image", "soil_crop_mask")}'
    crop_mask = np.load(crop_mask_file)
    axs[i + 2].imshow(crop_mask, alpha=0.9)
    axs[i + 2].set_title(os.path.basename(crop_mask_file), fontsize=60)
    axs[i + 2].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Create a 3x3 grid of subplots
fig, axs = plt.subplots(3, 3, figsize=(50, 50))

# Image directory
image_dir = '/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/'

# Import and display 3 random raw images
for i in range(3):
    # Select a random image file
    image_file = random.choice(glob.glob(os.path.join(image_dir, '*image.npy')))
    filename = os.path.basename(image_file)

    # Load and display the image in the top row
    im = np.load(image_file)
    axs[0, i].imshow(im, alpha=0.9)
    axs[0, i].set_title(filename, fontsize=60)
    axs[0, i].axis("off")

    # Find the corresponding soil crop mask file
    crop_mask_file = image_file.replace("image.npy", "soil_crop_mask.npy")
    crop_mask = np.load(crop_mask_file)

    # Display the soil crop mask in the second row with the same colormap as the overlaid image
    axs[1, i].imshow(crop_mask, cmap='viridis', alpha=0.9)  # Use 'viridis' colormap
    axs[1, i].set_title(os.path.basename(crop_mask_file), fontsize=60)
    axs[1, i].axis("off")

    # Overlay the crop mask on the base image in the third row
    masked_image = im.copy()
    masked_image[crop_mask == 1] = [255, 0, 0]  # Highlight the masked region in red

    # Display the overlaid image in the third row
    axs[2, i].imshow(masked_image, alpha=0.9)
    axs[2, i].set_title("Overlaid Image", fontsize=60)
    axs[2, i].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Create a custom colormap with yellow-green and dark blue
colors = ['#1f77b4', '#2ca02c']  # Dark blue and yellow-green
cmap = ListedColormap(colors)

# Create a 3x3 grid of subplots
fig, axs = plt.subplots(3, 3, figsize=(50, 50))

# Image directory
image_dir = '/scratch/pawsey0149/mmckay/raw_fullsize_images_masks/'

# Import and display 3 random raw images
for i in range(3):
    # Select a random image file
    image_file = random.choice(glob.glob(os.path.join(image_dir, '*image.npy')))
    filename = os.path.basename(image_file)

    # Load and display the image in the top row
    im = np.load(image_file)
    axs[0, i].imshow(im, alpha=0.9)
    axs[0, i].set_title(filename, fontsize=60)
    axs[0, i].axis("off")

    # Find the corresponding soil crop mask file
    crop_mask_file = image_file.replace("image.npy", "soil_crop_mask.npy")
    crop_mask = np.load(crop_mask_file)

    # Display the soil crop mask in the second row with the custom colormap
    axs[1, i].imshow(crop_mask, cmap=cmap, alpha=0.9)
    axs[1, i].set_title(os.path.basename(crop_mask_file), fontsize=60)
    axs[1, i].axis("off")

    # Overlay the crop mask on the base image in the third row
    masked_image = im.copy()
    masked_image[crop_mask == 1] = [255, 0, 0]  # Highlight the masked region in red

    # Display the overlaid image in the third row
    axs[2, i].imshow(masked_image, alpha=0.9)
    axs[2, i].set_title("Overlaid Image", fontsize=60)
    axs[2, i].axis("off")

plt.tight_layout()
plt.show()